# Jonathan Halverson
# Monday, May 9, 2016
# Spark SQL

In [1]:
#from pyspark.sql import SQLContext
#sqlCtx = SQLContext(sc)

In [2]:
from pyspark.sql import HiveContext
from pyspark.sql import Row

In [3]:
hiveCtx = HiveContext(sc)
lines = hiveCtx.read.json("boston_university.json")
lines.registerTempTable("events")
topTweets = hiveCtx.sql("""SELECT date_time, location, credit_url FROM events LIMIT 10""")

In [4]:
type(lines)

pyspark.sql.dataframe.DataFrame

In [5]:
type(topTweets)

pyspark.sql.dataframe.DataFrame

In [6]:
topTweets.show()

+--------------------+--------------------+--------------------+
|           date_time|            location|          credit_url|
+--------------------+--------------------+--------------------+
|All day on Monday...|Omni Hilton Head ...|http://www.bu.edu...|
|All day  every da...|                null|http://www.bu.edu...|
|10:30 am on Monda...|                null|http://www.bu.edu...|
|12:00 pm to 1:30 ...|  121 Bay State Road|http://www.bu.edu...|
|12:00 am   8:00 p...|                null|http://www.bu.edu...|
|10:00 am to 2:00 ...|GSU, 775 Commonwe...|http://www.bu.edu...|
|11:30 am   12:30 ...|                null|http://www.bu.edu...|
|12:00 pm to 1:30 ...|121 Bay State Roa...|http://www.bu.edu...|
|12:00 pm   5:00 p...|                null|http://www.bu.edu...|
|12:00 pm to 1:30 ...|                null|http://www.bu.edu...|
+--------------------+--------------------+--------------------+



Below we create a RDD from a DataFrame:

In [7]:
modified_url = topTweets.rdd.map(lambda x: (x[0], len(x[2])))
modified_url.collect()

[(u'All day on Monday, April 18, 2016', 57),
 (u'All day  every day until Sunday, April 24, 2016', 57),
 (u'10:30 am on Monday, April 18, 2016', 57),
 (u'12:00 pm to 1:30 pm  on Monday, April 18, 2016', 57),
 (u'12:00 am   8:00 pm  every day until Friday, April 22, 2016', 57),
 (u'10:00 am to 2:00 pm  on Tuesday, April 19, 2016', 57),
 (u'11:30 am   12:30 pm  on every Tuesday until Tuesday, May 3, 2016', 57),
 (u'12:00 pm to 1:30 pm  on Tuesday, April 19, 2016', 57),
 (u'12:00 pm   5:00 pm  on every Tuesday, Wednesday, Thursday, Friday, Saturday, & Sunday until Saturday, May 7, 2016',
  57),
 (u'12:00 pm to 1:30 pm  on Tuesday, April 19, 2016', 57)]

In [8]:
topTweets.select(["date_time"]).show()

+--------------------+
|           date_time|
+--------------------+
|All day on Monday...|
|All day  every da...|
|10:30 am on Monda...|
|12:00 pm to 1:30 ...|
|12:00 am   8:00 p...|
|10:00 am to 2:00 ...|
|11:30 am   12:30 ...|
|12:00 pm to 1:30 ...|
|12:00 pm   5:00 p...|
|12:00 pm to 1:30 ...|
+--------------------+



In [9]:
topTweets.filter(topTweets["location"] != 'null').select("location").show()

+--------------------+
|            location|
+--------------------+
|Omni Hilton Head ...|
|  121 Bay State Road|
|GSU, 775 Commonwe...|
|121 Bay State Roa...|
+--------------------+



In [10]:
hiveCtx.sql("""SELECT location FROM events WHERE location != "null" LIMIT 10""").show()

+--------------------+
|            location|
+--------------------+
|Omni Hilton Head ...|
|  121 Bay State Road|
|GSU, 775 Commonwe...|
|121 Bay State Roa...|
|3rd Floor Dining ...|
|Photonics Buildin...|
| Ziskind Lounge, GSU|
|International Cen...|
|The Towers, Frank...|
+--------------------+



In [11]:
topTweets.printSchema()

root
 |-- date_time: string (nullable = true)
 |-- location: string (nullable = true)
 |-- credit_url: string (nullable = true)



Below we create a RDD from a Python list and then create a DataFrame:

In [12]:
recs = [["Frank", 45, 180.4], ["Rocco", 23, 212.0], ["Claude", 38, 112.9]]
my_rdd = sc.parallelize(recs)

In [13]:
df = hiveCtx.createDataFrame(my_rdd)
df.show()

+------+---+-----+
|    _1| _2|   _3|
+------+---+-----+
| Frank| 45|180.4|
| Rocco| 23|212.0|
|Claude| 38|112.9|
+------+---+-----+



In [14]:
df.select("_2").show()

+---+
| _2|
+---+
| 45|
| 23|
| 38|
+---+



Below we create a DataFrame from a Python list of Row objects:

In [15]:
recsRow = [Row(name="Frank", age=45, weight=180.4),
           Row(name="Rocco", age=23, weight=212.0),
           Row(name="Claude", age=38, weight=112.9)]

In [16]:
df2 = hiveCtx.createDataFrame(recsRow)
df2.show()

+---+------+------+
|age|  name|weight|
+---+------+------+
| 45| Frank| 180.4|
| 23| Rocco| 212.0|
| 38|Claude| 112.9|
+---+------+------+



In [17]:
df2.select("name", df2.age + 1).show()

+------+---------+
|  name|(age + 1)|
+------+---------+
| Frank|       46|
| Rocco|       24|
|Claude|       39|
+------+---------+



In [18]:
df2.registerTempTable("Guys")
hiveCtx.sql("""SELECT name, age + 1 FROM Guys""").show()

+------+---+
|  name|_c1|
+------+---+
| Frank| 46|
| Rocco| 24|
|Claude| 39|
+------+---+

